In [50]:
import requests
import shutil
import os.path
import pandas as pd
import time
import json
import random

import nest_asyncio

nest_asyncio.apply()
import asyncio
import aiohttp

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
]

In [51]:
def get_data(page_size):
    page = requests.get(
        f"https://eismoinfo.lt/eismoinfo-backend/camera-info-table/4044?pageNumber=0&pageSize={page_size}"
    ).content.decode()
    df = pd.DataFrame(json.loads(page).get("info"))
    return df


df = get_data(page_size=1)
df.head()

,id,date,image
0,186305502,1684262201000,https://www.eismoinfo.lt/eismoinfo-backend/ima...


In [53]:
async def download_file(session, url, filename):
    async with session.get(
        url, headers={"User-Agent": random.choice(user_agents)}
    ) as response:
        with open(filename, "wb") as f:
            while True:
                chunk = await response.content.read(1024)
                if not chunk:
                    break
                f.write(chunk)
    # Add random sleep time between requests
    await asyncio.sleep(random.uniform(1, 3))


async def main():
    urls = df["image"].tolist()
    photo_ids = df["id"].tolist()
    # Set up aiohttp client session
    async with aiohttp.ClientSession() as session:
        # Create list of async tasks
        tasks = []
        for photo_id, url in zip(photo_ids, urls):
            filename = os.path.join(
                os.path.abspath(""), "Photos_for_model\\%s.jpg" % photo_id
            )
            task = asyncio.create_task(download_file(session, url, filename))
            tasks.append(task)
        # Run async tasks in parallel
        await asyncio.gather(*tasks)


asyncio.run(main())